In [3]:
#for delhi   //    fixing the bugs 

import requests
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
from selenium.common.exceptions import TimeoutException, WebDriverException
import pandas as pd

driver = webdriver.Chrome()

def get_pdf_links(url):
    driver.get(url)
    view_links = []
    try:
        view_tags = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, '//a[text()="View..."]')))
        for tag in view_tags:
            view_links.append(tag.get_attribute('href'))
    except Exception as e:
        print(f"Error: {e}")
    return view_links

def download_pdf(pdf_url, folder):
    try:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            if 'ayayaya' in pdf_url.lower():
                filename = re.split('[Aa]', pdf_url)[-1]
            else:
                filename = pdf_url.split('/')[-1]
       
            filename = os.path.basename(filename)
            filename = os.path.join(folder, filename)
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
            return filename
        else:
            print(f"Failed to download: {pdf_url}")
            return None
    except Exception as e:
        print(f"Error downloading {pdf_url}: {e}")
        return None

def scrape_and_save_metadata(url):
    metadata_list = []
    try:
        driver.get(url)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "table")))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        acts_table = soup.find('table', class_='table')
        acts_rows = acts_table.find_all('tr')

        act_counter = {}

        for row in acts_rows[1:]:  
            columns = row.find_all('td')
            enactment_date = columns[0].get_text(strip=True) if columns[0].get_text(strip=True) else "N/A"
            act_number = columns[1].get_text(strip=True) if columns[1].get_text(strip=True) else "N/A"
            short_title = columns[2].get_text(strip=True) if columns[2].get_text(strip=True) else "N/A"

            year = enactment_date.split('-')[-1] if enactment_date != "N/A" else "N/A"
            
            act_counter[year] = act_counter.get(year, 0) + 1
            act_identifier = act_counter[year]
            
            metadata_list.append({'Enactment Date': enactment_date, 'Act Number': act_number, 'Short Title': short_title, 'Type': 'state'})

        print("Metadata scraped")
    except TimeoutException as e:
        print("Timeout occurred while scraping page:", url)
    except WebDriverException as e:
        print("WebDriverException occurred:", e)
    except Exception as e:
        print("An error occurred:", e)
    
    metadata_df = pd.DataFrame(metadata_list)  
    return metadata_df 

def get_direct_pdf_links(pdf_page_urls):
    direct_pdf_links = []
    os.makedirs('downloaded_pdfs_delhi', exist_ok=True)  
    for url in pdf_page_urls:   # setting the limit
        driver.get(url)
        try:
            pdf_link_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'row'))).find_element(By.TAG_NAME, 'a')
            pdf_link = pdf_link_element.get_attribute('href')
            if pdf_link:
                filename = pdf_link.split('/')[-1]  
                direct_pdf_links.append(filename)
                print(f"PDF Download Link: {filename}")
                download_pdf(pdf_link, 'downloaded_pdfs_delhi')  
            else:
                print("Failed to extract PDF link under 'row' class for:", url)
        
                pdf_link_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'container'))).find_element(By.TAG_NAME, 'a')
                pdf_link = pdf_link_element.get_attribute('href')
                if pdf_link:
                    filename = pdf_link.split('/')[-1]  
                    direct_pdf_links.append(filename)
                    print(f"PDF Download Link: {filename}")
                    download_pdf(pdf_link, 'downloaded_pdfs_delhi')  
                else:
                    print("Failed to extract PDF link under 'container' class for:", url)
                   
        except TimeoutException:
            print(f"Timeout occurred while retrieving PDF link for: {url}")
           
        except Exception as e:
            print(f"Error occurred while retrieving PDF link for {url}: {e}")    
           
    return direct_pdf_links



def main():
    base_url = 'https://www.indiacode.nic.in/handle/123456789/2517/simple-search?page-token=b5a9925d3e48&page-token-value=f49501cf7031adb27c3473c2c32521e4&nccharset=6FB497B8&location=123456789%2F2517&query=&rpp=500&sort_by=score&order=DESC&submit_search=Update'

    metadata_df = scrape_and_save_metadata(base_url)
    metadata_df.to_csv('metadata_from_link_delhi.csv', index=False)

    pdf_page_urls = get_pdf_links(base_url)
    direct_pdf_links = get_direct_pdf_links(pdf_page_urls)

    if len(metadata_df) == len(direct_pdf_links):
        metadata_df['PDF Link'] = direct_pdf_links
        metadata_df.to_csv('combined_metadata_maharashtra_test.csv', index=False)
    else:
        print("Lengths of metadata and PDF links do not match. Unable to combine.")
    
if __name__ == "__main__":
    main()

driver.quit()


Metadata scraped
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Download Link: 2517#
Downloaded: downloaded_pdfs_delhi/2517#
PDF Down